In [ ]:
import sys
!{sys.executable} -m pip install --user scikit-allel

In [1]:
import numpy as np
import scipy
import pandas
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('white')
sns.set_style('ticks')
sns.set_context('notebook')
import h5py
import allel; print('scikit-allel', allel.__version__)

scikit-allel 1.3.8


### create a filtered VCF containing only invariant sites
vcftools --gzvcf "$OUTDIR/$INPUT" \
        --max-maf 0 \
        --recode --recode-INFO-all --stdout | bgzip -c > "$OUTDIR/$INVARIANT"

### create a filtered VCF containing only variant sites
vcftools --gzvcf "$OUTDIR/$INPUT" \
        --mac 1 \
        --recode --recode-INFO-all --stdout | bgzip -c > "$OUTDIR/$VARIANT"

### index both vcfs using tabix
tabix -f "$OUTDIR/$INVARIANT"
tabix -f "$OUTDIR/$VARIANT"

## Get data

In [ ]:
callset_fn = '/users/mcevoysu/scratch/output/Aalba_random/jupytertest/Aalba_random_SPET_allsites.h5'
callset_all = h5py.File(callset_fn, mode='r')

In [ ]:
callset_invar_fn = '/users/mcevoysu/scratch/output/Aalba_random/jupytertest/Aalba_random_SPET_invariant.h5'
callset_invar = h5py.File(callset_invar_fn, mode='r')

## Make datasets

In [ ]:
allsites = allel.VariantChunkedTable(callset_all['variants'])
invariants = allel.VariantChunkedTable(callset_invar['variants'])
variants = allel.VariantChunkedTable(callset_var['variants'])
variants_np = variants[:]
invariants = invariants[:]
rawsnps = variants_np.query('(is_snp == True)')
rawindels = variants_np.query('(is_snp != True)')

In [ ]:
invariants

## Make biallelic SNP dataset

In [ ]:
filter_expression = '(numalt == 1)'
biallelic_snps = rawsnps.query(filter_expression)[:]

## Plot function

In [ ]:
def plot_hist(f, dsubset='', bins=30 ):
    if dsubset == 'invar':
        x = invariants[f][:]
        l = 'Invariant'
    elif dsubset == 'var':
        x = variants[f][:]
        l = 'Variant'
    elif dsubset == 'snp':
        x = rawsnps[f][:]
        l = 'Raw SNP'
    elif dsubset == 'indel':
        x = rawindels[f][:]
        l = 'Raw Indel'
    elif dsubset == 'sel':
        x = snp_selection[f][:]
        l = 'Filtered SNP'
    elif dsubset == 'biallelic':
        x = biallelic_snps[f][:]
        l = 'Biallelic SNP'
    else:
        x = allsites[f][:]
        l = 'Allsites'
    fig, ax = plt.subplots(figsize=(10, 5))
    sns.despine(ax=ax, offset=10)
    ax.hist(x, bins=bins)
    ax.set_xlabel(f)
    ax.set_ylabel('No. variants')
    ax.set_title('%s %s distribution' % (l, f))

## QD - Variant Confidence/Quality by Depth

In [ ]:
plot_hist('QD','biallelic') # Variant Confidence/Quality by Depth

In [ ]:
filter_expression = '(QD >= 2.0)'
snp_selection = rawsnps.query(filter_expression)[:]

In [ ]:
plot_hist('QD','sel') # Variant Confidence/Quality by Depth

In [ ]:
plot_hist('QD','invar') # Variant Confidence/Quality by Depth

In [ ]:
filter_expression = '(QD >= 0.0)'
invar_selection = invariants.query(filter_expression)[:]

In [ ]:
invar_selection

In [ ]:
invariants[0:3]

In [ ]:
plot_hist('QD','indel') # Variant Confidence/Quality by Depth

## MQ - RMS mapping quality

In [ ]:
plot_hist('MQ','biallelic')

In [ ]:
filter_expression = '(MQ >= 40.0)'
snp_selection = rawsnps.query(filter_expression)[:]

In [ ]:
plot_hist('MQ','invar')

In [ ]:
plot_hist('MQ','indel')

## SOR - Symmetric Odds Ratio of 2x2 contingency table to detect strand bias

In [ ]:
plot_hist('SOR') # Symmetric Odds Ratio of 2x2 contingency table to detect strand bias

In [ ]:
plot_hist('SOR','snp') # Symmetric Odds Ratio of 2x2 contingency table to detect strand bias

In [ ]:
plot_hist('SOR','indel') # Symmetric Odds Ratio of 2x2 contingency table to detect strand bias

In [ ]:
plot_hist('SOR','invar') # Symmetric Odds Ratio of 2x2 contingency table to detect strand bias

In [ ]:
invariants['SOR'][:]

In [ ]:
filter_expression = '(SOR >= 0)'
invarSOR = invariants.query(filter_expression)[:]
invarSOR

In [ ]:
np.count_nonzero(invarSOR)

## MQRankSum - Z-score From Wilcoxon rank sum test of Alt vs. Ref read mapping qualities

In [ ]:
plot_hist('MQRankSum') # Z-score From Wilcoxon rank sum test of Alt vs. Ref read mapping qualities

In [ ]:
plot_hist('MQRankSum','snp') # Z-score From Wilcoxon rank sum test of Alt vs. Ref read mapping qualities

In [ ]:
plot_hist('MQRankSum','indel') # Z-score From Wilcoxon rank sum test of Alt vs. Ref read mapping qualities

In [ ]:
plot_hist('MQRankSum','invar') # Z-score From Wilcoxon rank sum test of Alt vs. Ref read mapping qualities

## ReadPosRankSum - Z-score from Wilcoxon rank sum test of Alt vs. Ref read position bias

In [ ]:
plot_hist('ReadPosRankSum') # Z-score from Wilcoxon rank sum test of Alt vs. Ref read position bias

In [ ]:
plot_hist('ReadPosRankSum','snp') # Z-score from Wilcoxon rank sum test of Alt vs. Ref read position bias

In [ ]:
plot_hist('ReadPosRankSum','indel') # Z-score from Wilcoxon rank sum test of Alt vs. Ref read position bias

In [ ]:
plot_hist('ReadPosRankSum','invar') # Z-score from Wilcoxon rank sum test of Alt vs. Ref read position bias

## DP - Approximate read depth

In [ ]:
plot_hist('DP')

In [ ]:
plot_hist('DP','snp')

In [ ]:
plot_hist('DP','biallelic')

In [ ]:
plot_hist('DP','indel')

In [ ]:
plot_hist('DP','invar')

## AN - Total number of alleles in called genotypes

In [ ]:
plot_hist('AN') # Total number of alleles in called genotypes

In [ ]:
plot_hist('AN','snp') # Total number of alleles in called genotypes

In [ ]:
plot_hist('AN','indel') # Total number of alleles in called genotypes

In [ ]:
plot_hist('AN','invar') # Total number of alleles in called genotypes

## Filter tests

In [ ]:
filter_expression = '(QD > 2)'
snp_selection = biallelic_snps.query(filter_expression)[:]
snp_selection

In [ ]:
snp_selection

In [ ]:
plot_hist('QD','snp')

In [ ]:
plot_hist('QD','sel')

In [ ]:
rawsnps['DP'].mean()

In [ ]:
biallelic_snps['DP'].mean()

In [ ]:
filter_expression = '(DP > 20000) & (DP < 40000)'
snp_selection = biallelic_snps.query(filter_expression)[:]
snp_selection

In [ ]:
plot_hist('DP','snp')

In [ ]:
plot_hist('DP','sel')

In [ ]:
snp_selection['DP'].mean()

In [ ]:
qd = variants['QD'][:]
np.count_nonzero(np.isnan(qd))

In [ ]:
filter_expression = '(DP > 20)'
variant_selection = variants.eval(filter_expression)[:]
np.count_nonzero(variant_selection)

## Selected filter

In [ ]:
# QD: Variant Confidence/Quality by Depth
# AN: Total number of alleles in called genotypes
filter_expression = '(DP > 20) & (DP < 40000) & (QD > 2) & (AN > 576) & (is_snp)'
variant_selection = variants.eval(filter_expression)[:]
np.count_nonzero(variant_selection)

In [ ]:
filter_expression = '(QD >= 0.0)'
invariant_selection = invariants.eval(filter_expression)[:]

In [ ]:
variant_selection[:]

## Genotype

In [ ]:
calldata_invar = callset_invar['calldata']
list(calldata_invar)

In [ ]:
calldata_invar[0]

In [ ]:
genotypes_invar = allel.GenotypeChunkedArray(calldata_invar['GT'])
genotypes_invar

In [ ]:
# using the selected filters set above
gt_filtered_invar = genotypes_invar.subset(invariant_selection)
gt_filtered_invar[0:1]

In [ ]:
# grab the allele counts for the populations
ac = gt_filtered_snps.count_alleles()
ac

In [ ]:
ac[:]

In [ ]:
# Which ones are biallelic?
is_biallelic_01 = ac.is_biallelic_01()[:]
ac1 = ac.compress(is_biallelic_01, axis=0)[:, :2]
ac1
##this part of the code is only for graphing the SFS, is not useful for filtering biallelic sites

In [ ]:
# plot the sfs of the derived allele
s = allel.sfs_folded(ac1)
allel.plot_sfs(s, yscale="linear", n=ac1.sum(axis=1).max())

In [ ]:
biallelic = (ac.max_allele() == 1)
###This is the filter expression for biallelic sites
biallelic

In [ ]:
# select only the biallelic variants
gt_biallelic = gt_filtered_snps.compress(biallelic)
gt_biallelic

In [ ]:
n_variants = len(gt_biallelic)
n_variants

In [ ]:
pc_missing = gt_biallelic.count_missing(axis=0)[:] * 100 / n_variants
pc_het = gt_biallelic.count_het(axis=0)[:] * 100 / n_variants

In [ ]:
gt_biallelic

## Samples

In [ ]:
samples_var = callset_var['samples']
samples_var = list(samples_var)
samples_var

In [ ]:
samples_var[0:4]

In [ ]:
samples_var[-4:]

In [ ]:
samples_fn = '~/scratch/data/Aalba/aalba_sample_list-scikit-allel.txt'
samples = pandas.read_csv(samples_fn, sep='\t')
samples

In [ ]:
samples.Population.value_counts()

In [ ]:
populations = samples.Population.unique()
populations
###This identifiers come from the metadata file

## Gt frequency function

In [ ]:
def plot_genotype_frequency(pc, title):
    fig, ax = plt.subplots(figsize=(24, 5))
    sns.despine(ax=ax, offset=24)
    left = np.arange(len(pc))
    palette = sns.color_palette("hls", 24)
    pop2color = {'AUT00179': palette[7],
                 'ITA00271': palette[1],
                 'SVN00025': palette[2],
                 'SVN00023': palette[3],
                 'ITA00260': palette[22],
                 'ROU00358': palette[5],
                 'ROU00104': palette[6],
                 'ROU00389': palette[0],
                 'ROU00477': palette[8],
                 'AUT00215': palette[9],
                 'DEU00114': palette[10],
                 'FRA00006': palette[11],
                 'ITA00069': palette[12],
                 'ITA00029': palette[13],
                 'ITA00217': palette[14],
                 'FRA00019': palette[15],
                 'ESP00339': palette[16],
                 'FRA00004': palette[17]}
    colors = [pop2color[p] for p in samples.Population]
    ax.bar(left, pc, color=colors)
    ax.set_xlim(0, len(pc))
    ax.set_xlabel('Sample index')
    ax.set_ylabel('Percent calls')
    ax.set_title(title)
    handles = [mpl.patches.Patch(color=palette[7]),
               mpl.patches.Patch(color=palette[1]),
               mpl.patches.Patch(color=palette[2]),
               mpl.patches.Patch(color=palette[3]),
               mpl.patches.Patch(color=palette[22]),
               mpl.patches.Patch(color=palette[5]),
               mpl.patches.Patch(color=palette[6]),
               mpl.patches.Patch(color=palette[0]),
               mpl.patches.Patch(color=palette[8]),
               mpl.patches.Patch(color=palette[9]),
               mpl.patches.Patch(color=palette[10]),
               mpl.patches.Patch(color=palette[11]),
               mpl.patches.Patch(color=palette[12]),
               mpl.patches.Patch(color=palette[13]),
               mpl.patches.Patch(color=palette[14]),
               mpl.patches.Patch(color=palette[15]),
               mpl.patches.Patch(color=palette[16]),
               mpl.patches.Patch(color=palette[17])]
    ax.legend(handles=handles, labels=['AUT00179', 'ITA00271', 'SVN00025', 'SVN00023', 'ITA00260',
       'ROU00358', 'ROU00104', 'ROU00389', 'ROU00477', 'AUT00215',
       'DEU00114', 'FRA00006', 'ITA00069', 'FRA00019', 'FRA00004',
       'ESP00339', 'ITA00217', 'ITA00029'], title='Population',
              bbox_to_anchor=(1, 1), loc='upper left')

## Plot missing

In [ ]:
plot_genotype_frequency(pc_missing, 'Missing')

In [ ]:
# sample with highest amount of missing data
np.argsort(pc_missing)[-19:]

In [ ]:
pc_missing

In [ ]:
samples[345:346]

In [ ]:
samples.ID[345]

## Plot heterozygosity

In [ ]:
plot_genotype_frequency(pc_het, 'Heterozygosity')

In [ ]:
pc_het[247]

In [ ]:
gt_biallelic[247]

## PCA

In [ ]:
palette = sns.color_palette("hls",23)
pop_colours = {    
                'AUT00179': palette[7],
                 'ITA00271': palette[1],
                 'SVN00025': palette[2],
                 'SVN00023': palette[3],
                 'ITA00260': palette[22],
                 'ROU00358': palette[5],
                 'ROU00104': palette[6],
                 'ROU00389': palette[0],
                 'ROU00477': palette[8],
                 'AUT00215': palette[9],
                 'DEU00114': palette[10],
                 'FRA00006': palette[11],
                 'ITA00069': palette[12],
                 'ITA00029': palette[13],
                 'ITA00217': palette[14],
                 'FRA00019': palette[15],
                 'ESP00339': palette[16],
                 'FRA00004': palette[17]
}

In [ ]:
def plot_pca_coords(coords, model, pc1, pc2, ax, sample_population):
    sns.despine(ax=ax, offset=5)
    x = coords[:, pc1]
    y = coords[:, pc2]
    for pop in populations:
        flt = (sample_population == pop)
        ax.plot(x[flt], y[flt], marker='o', linestyle=' ', color=pop_colours[pop], 
                label=pop, markersize=6, mec='k', mew=.5)
    ax.set_xlabel('PC%s (%.1f%%)' % (pc1+1, model.explained_variance_ratio_[pc1]*100))
    ax.set_ylabel('PC%s (%.1f%%)' % (pc2+1, model.explained_variance_ratio_[pc2]*100))
    

def fig_pca(coords, model, title, sample_population=None):
    if sample_population is None:
        sample_population = samples.Population
    # plot coords for PCs 1 vs 2, 3 vs 4
    fig = plt.figure(figsize=(10, 5))
    ax = fig.add_subplot(1, 2, 1)
    plot_pca_coords(coords, model, 0, 1, ax, sample_population)
    ax = fig.add_subplot(1, 2, 2)
    plot_pca_coords(coords, model, 2, 3, ax, sample_population)
    ax.legend(bbox_to_anchor=(1, 1), loc='upper left')
    fig.suptitle(title, y=1.02)
    fig.tight_layout()

In [ ]:
ac2 = gt_biallelic.count_alleles()
ac2

In [ ]:
flt = (ac2[:, :2].min(axis=1) > 1)
gf = gt_biallelic.compress(flt, axis=0)
gn = gf.to_n_alt()
gn

In [ ]:
coords1, model1 = allel.pca(gn, n_components=10, scaler='patterson')

In [ ]:
fig_pca(coords1, model1, 'Figure 1. Conventional PCA.')